## Setup 

In [227]:
import os
import random
import itertools

startingClasses = ["Warrior","Knight","Wanderer","Thief","Bandit","Hunter",
                   "Sorcerer","Pyromancer","Cleric", "Deprived"]

startingGifts = ["None","Divine Blessing","Black Firebomb","Twin Humanity",
                 "Binoculars","Pendant","Master Key","Tiny Being's Ring",
                 "Old Witch's Ring"]

allBosses = [
    "Moonlight Butterfly", "Pinwheel", "Taurus Demon", "Quelaag", "Bell Gargoyles",
    "Iron Golem", "Ceaseless Discharge", "Ornstein and Smough", "Four Kings",
    "Seath", "Bed of Chaos", "Nito", "Kalameet", "Artorias", "Manus", "Gwyn"]

npcOptions = ["Kill", "Talk to"]


cPath = "/home/federico/Nextcloud/Python/scripts/Games/DarkSouls_Minigames/Challenges"

# #### Google Drive setup
# from google.colab import drive
# drive.mount('/content/drive')
# notebook_dir = "/content/drive/MyDrive/Games/DarkSouls/DarkSouls_Minigames"
# cPath = os.path.join(notebook_dir, "Challenges")




boss_challenge_path = f"{cPath}/boss_challenge.txt"
bosses_path = f"{cPath}/bosses.txt"
boss_weapons_path = f"{cPath}/boss_weapons.txt"
covenants_path = f"{cPath}/covenants.txt"
npcs_path = f"{cPath}/npcs.txt"
recover_path = f"{cPath}/recover.txt"
weapon_type_path = f"{cPath}/weapon_type.txt"
stats_path = f"{cPath}/stats.txt"

boardSize = 5  #### 5x5
entries = boardSize * boardSize 


#### Functions & file reading

In [ ]:
def load_list(path):
    """Return list of meaningful lines from a newline-separated txt file."""
    with open(path, "r") as f:
        return [line.strip() for line in f if line.strip()]


boss_challenges = load_list(boss_challenge_path)
weapon_types    = load_list(weapon_type_path)
stats_list      = load_list(stats_path)
npc_list        = load_list(npcs_path)
covenants_list = load_list(covenants_path)
recover_list   = load_list(recover_path)
boss_weapons = load_list(boss_weapons_path)

boss_challenge_cycle = itertools.cycle(boss_challenges)
weapon_type_cycle    = itertools.cycle(weapon_types)


def random_stat_challenge(stats_pool):
    stat = random.choice(stats_pool)
    value = random.randint(20, 40)
    return f"{stat} → {value}"

def random_level_cap():
    cap = random.randint(20, 60)
    return f"Level Cap → {cap}"

def random_covenant_task(covenants):
    covenant = random.choice(covenants)
    return f"Join {covenant}"

def random_recover_task(recover_list):
    return random.choice(recover_list)


## Generating challenges

In [229]:
final_challenges = []

challenge_bosses = 3   #### number of bosses that get a challenge
standard_bosses  = 2   #### number of bosses that remain plain

all_bosses_copy = allBosses.copy()
random.shuffle(all_bosses_copy)

challenged_bosses = all_bosses_copy[:challenge_bosses]

plain_bosses = all_bosses_copy[challenge_bosses:challenge_bosses + standard_bosses]


for boss in challenged_bosses: #### Assign random variants to challenged bosses
    variants = [
        f"{boss} - {next(boss_challenge_cycle)}",
        f"{boss} - {next(weapon_type_cycle)}",
    ]
    final_challenges.append(random.choice(variants))


final_challenges.extend(plain_bosses)


In [230]:

#### Add 1 non-duplicate stat challenge
stat_challenge = random_stat_challenge(stats_list)
final_challenges.append(stat_challenge)


final_challenges.append(random_level_cap())

for _ in range(2): #### 2 challenges for npcs
    final_challenges.append(random_npc_task(npc_list, npcOptions))

for _ in range(2): #### 2 challenges for covenants
    final_challenges.append(random_covenant_task(covenants_list))


#### Boss weapons
weapon = random.choice(boss_weapons)
final_challenges.append(f"Forge {weapon}")


npc_challenges = []
for npc in npc_list:
    action = random.choice(["Kill", "Talk to"])   # random every iteration
    npc_challenges.append(f"{action} {npc}")




#### Check for total length

In [231]:
#### Fill remaining slots to reach the total board size
current_entries = len(final_challenges)
remaining_slots = entries - current_entries

#### Use recover tasks to fill the remaining slots
for _ in range(remaining_slots):
    final_challenges.append(random_recover_task(recover_list))


#### Shuffle & print

In [232]:
random.shuffle(final_challenges)
random.shuffle(final_challenges)

final_challenges = final_challenges[: entries ]

print(f'Starting class: {random.choice(startingClasses)} with {random.choice(startingGifts)} {len(final_challenges)} entries\n')

for i in range(0, len(final_challenges), boardSize):
    row = final_challenges[i:i+boardSize] #### board size to copy-paste on spreadsheet
    print("\t".join(row))

Starting class: Bandit with Twin Humanity 25 entries

Join Darkwraith	Knight Set	Talk to Chester	Quelaag	Level Cap → 53
Iron Flesh	HUM → 24	Kalameet - Mid Roll	Evolve a shield	Replenishment
Undead Asylum F2 West Key	Royal helm	Sniper Crossbow	Gwyn - Greatswords	Rare Ring of Sacrifice
Ring of the Evil Eye	Tranquil Walk of Peace	Forge Darkmoon Bow	Join Gravelord Servant	Four Kings
Hawk Ring	Kill Blacksmith Andre	Ceaseless Discharge - Fast Roll	Cloranthy Ring	Sewer Chamber Key


In [233]:
output_file = f"{cPath}/final_challenge_board.txt"

with open(output_file, "w") as f:
    for i in range(0, len(final_challenges), boardSize):
        row = final_challenges[i:i+boardSize]
        f.write("\t".join(row) + "\n")

print(f"Final challenge board written to: {output_file}")


Final challenge board written to: /home/federico/Nextcloud/Python/scripts/Games/DarkSouls_Minigames/Challenges/final_challenge_board.txt


## Google drive - sheets integration

In [ ]:
# -----------------------------
# Dependencies (install if missing)
# -----------------------------
# !pip install gspread
# !pip install gspread-formatting
# from google.colab import auth
# import gspread
# from google.auth import default
# from gspread_formatting import *


In [ ]:
# -----------------------------
# Google Sheets authorization
# -----------------------------
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
from gspread_formatting import *
from datetime import datetime
import gspread.utils as gutils
import random

# -----------------------------
# Connect to Google Sheets
# -----------------------------
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet = gc.open("DS Bingo")

# -----------------------------
# Generate unique tab name (Run_YYYY-MM-DD_HH-MM)
# -----------------------------
base_name = "Run_" + datetime.now().strftime("%Y-%m-%d_%H-%M")
existing_titles = [ws.title for ws in spreadsheet.worksheets()]

if base_name not in existing_titles:
    tab_name = base_name
else:
    suffix = 2
    while f"{base_name}_{suffix}" in existing_titles:
        suffix += 1
    tab_name = f"{base_name}_{suffix}"

worksheet = spreadsheet.add_worksheet(title=tab_name, rows=str(boardSize), cols=str(boardSize))

# -----------------------------
# Fill the board
# -----------------------------
rows = [final_challenges[i:i+boardSize] for i in range(0, len(final_challenges), boardSize)]

# Resize sheet to fit the summary row
worksheet.resize(rows=len(rows) + 1, cols=boardSize)

# Update the challenge rows
worksheet.update(rows)

# -----------------------------
# ADD: append summary row (merged full-width)
# -----------------------------
summary_class = random.choice(startingClasses)
summary_gift = random.choice(startingGifts)
summary_text = (
    f"Starting class:   {summary_class}   with   {summary_gift}"
)

summary_row = len(rows) + 1
worksheet.resize(rows=summary_row, cols=boardSize)

# Write text in A
worksheet.update_cell(summary_row, 1, summary_text)

# Merge A:lastColumn on the summary row
last_col_letter = gutils.rowcol_to_a1(1, boardSize).rstrip("1")
merge_range = f"A{summary_row}:{last_col_letter}{summary_row}"

worksheet.merge_cells(merge_range)

# Optional: match the format of the other cells
format_cell_range(worksheet, merge_range, cell_fmt)


# -----------------------------
# Formatting
# -----------------------------
cell_fmt = cellFormat(
    backgroundColor=color(0.33, 0.33, 0.33),
    textFormat=textFormat(bold=False, foregroundColor=color(1, 1, 1), fontSize=15),
    horizontalAlignment='CENTER',
    verticalAlignment='MIDDLE',
    wrapStrategy='WRAP',
    borders=Borders(
        top=Border('SOLID', color=color(1,1,1)),
        bottom=Border('SOLID', color=color(1,1,1)),
        left=Border('SOLID', color=color(1,1,1)),
        right=Border('SOLID', color=color(1,1,1)),
    )
)

# Format the whole challenge grid
end_cell = gutils.rowcol_to_a1(len(rows), boardSize)
format_cell_ranges(worksheet, [(f"A1:{end_cell}", cell_fmt)])

# Format summary row (Cell A_<last>)
format_cell_range(worksheet, f"A{summary_row_index}", cell_fmt)

# -----------------------------
# Set column width + row height
# -----------------------------
# Column widths
for c in range(1, boardSize+1):
    col_letter = gutils.rowcol_to_a1(1, c).rstrip("1")  # "A", "B", ...
    set_column_width(worksheet, f"{col_letter}:{col_letter}", 220)

# Row heights (include summary row)
for r in range(1, len(rows)+2):  # +1 for summary row
    set_row_height(worksheet, f"{r}:{r}", 120)

print(f"✔️ Challenge board saved to tab: {tab_name}")
print(f"✔️ Added summary: {summary_text}")
